<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Standard lib imports
import os
import csv
import glob
import time
import argparse
import warnings
import pandas as pd
import os.path as osp

In [2]:
!pip install horovod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 13.7 MB/s 
  Created wheel for horovod: filename=horovod-0.26.1-cp38-cp38-linux_x86_64.whl size=28370371 sha256=d8a8a6481e286f4168752a138cc286419731e4bb9385a0b0af82395663b53204
  Stored in directory: /root/.cache/pip/wheels/eb/0b/90/d53058f75f3ae3db9557f3e55dd8c016b2397e9b38557c8b66
Successfully built horovod


In [3]:
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import horovod.torch as hvd
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

In [4]:
!pip install modelmanager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for modelmanager: filename=ModelManager-0.1.2-py3-none-any.whl size=1997 sha256=4a6c6f1c2d168baf5958bea275ca93fce52eaee619f6f5d6e3cd89ab7fb91476
  Stored in directory: /root/.cache/pip/wheels/c0/a7/bd/95fd7a99b8046d504481e5734bda1594569e215ffe6ae7e715
Successfully built modelmanager


In [5]:
! git clone https://github.com/BCV-Uniandes/Bonet.git

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.


In [20]:
!cd Bonet

In [23]:
from main_3D_BoNet import BoNet

ModuleNotFoundError: ignored

In [22]:
# Local imports
from Models.bonet import BoNet
from data.boneage_loader import BoneageDataset

ModuleNotFoundError: ignored